# Using commuting projectors in STRUPHY

In STRUPHY the commuting projectors of the de Rham sequence are based on __inter-/histopolation__ of spline basis functions at Greville points. 

There are two kinds of projectors in STRUPHY:

* __global projectors__ based on inter-/histopolation in the __whole domain__ 
* __local projectors__ based on quasi-inter-/histopolation in a __neighbourhood of $x_i$__

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import time
from sys import getsizeof

import hylife.utilitis_FEEC.projectors_global as proj_glob
import hylife.utilitis_FEEC.projectors_global_fast as proj_glob_fast

import hylife.utilitis_FEEC.evaluation as eva
from hylife.utilitis_FEEC.linalg_kron import kron_matvec_3d


import hylife.utilitis_FEEC.bsplines as bsp

## Global projectors in 1D

In [ ]:
# ------------------------
# function to be projected
# ------------------------
fun = lambda xi1 : np.sin( np.pi*( np.sin(2.*np.pi*xi1) ) )

#-----------------
# Create the grid:
#-----------------
# side lengths of logical cube
L = 1.

# spline degree
p = 3   

# periodic boundary conditions (use 'False' if clamped)
bc = True

# errors:
before_max = 0.
before_L2  = 0.
before_L1  = 0.

# loop over different number of elements (convergence test)
Nel_cases = [8, 16, 32, 64, 128, 256, 512]

# loop over different number of quadrature points per element
Nq_cases = [6]

for Nel in Nel_cases:
    
    print('Nel=', Nel)
    
    # element boundaries
    el_b = np.linspace(0., L, Nel + 1) 

    # knot sequences
    T = bsp.make_knots(el_b, p, bc)
    
    for Nq in Nq_cases:

        # create an instance of the projector class
        obj = proj_glob.projectors_1d(T, p, bc, Nq)

        # compute coefficients
        #coeffs = obj.PI_0(fun)
        coeffs = obj.PI_1(fun)

        # grid points for error computation
        pts_loc, wts_loc = np.polynomial.legendre.leggauss(6)   

        pts, wts = bsp.quadrature_grid(el_b, pts_loc, wts_loc)    

        xgrid = pts.flatten()   # error grid

        # projection evaluated at the grid points
        f_h = eva.FEM_field_1d(coeffs, 1, xgrid, T, p, bc)

        # function evaluated at the grid points
        f   = fun( xgrid )

        # max-norm
        err_max = np.max( np.abs(f - f_h) )

        # L2-norm
        err_L2 = np.sqrt( np.sum( np.abs(f - f_h)**2 ) / np.size(f) )

        # L1-norm
        err_L1 = np.sum( np.abs(f - f_h) ) / np.size(f) 

        # print errors
        print( 'Nq=', Nq, 
               '{:10.7f}'.format(err_max), '{:10.7f}'.format(before_max/err_max), ' | ',
               '{:10.7f}'.format(err_L2),  '{:10.7f}'.format(before_L2/err_L2),   ' | ', 
               '{:10.7f}'.format(err_L1),  '{:10.7f}'.format(before_L1/err_L1),   ' | ')

        before_max = err_max
        before_L2  = err_L2
        before_L1  = err_L1
        
    #print('')
    
    
plt.plot(xgrid, f_h, 'ro', xgrid, f, 'b-')

## Global projectors in 3D 

In [ ]:

# ------------------------
# function to be projected
# ------------------------
fun_a = lambda xi1, xi2, xi3 : np.sin( np.pi*( np.sin(2.*np.pi*xi1)
                                              *np.sin(2.*np.pi*xi2)
                                              *np.sin(2.*np.pi*xi3) ) )

fun_b = lambda xi1, xi2, xi3 : np.sin( np.pi*( np.sin(2.*np.pi*(xi1-0.1))
                                              *np.sin(2.*np.pi*(xi2-0.2))
                                              *np.sin(2.*np.pi*(xi3-0.3)) ))

fun = lambda xi1, xi2, xi3 :   fun_a(xi1,xi2,xi3)*fun_b(xi1,xi2,xi3)

#-----------------
# Create the grid:
#-----------------
# side lengths of logical cube
L = [3., 2., 1.] 

# spline degrees
p = [3, 3, 3]   

# periodic boundary conditions (use 'False' if clamped)
bc = [True, True, True] 

# number of quadrature points per element
Nq_k = 6

# ------------------------
# loop over projector to test (0, 11, 12, 13, 21, 22, 23, 3):
# ------------------------
comp_cases = [0,11,12,13,21,22,23,3]


# loop over different number of elements (convergence test)
Nel_cases = [2**n for n in range(2,4)]

for comp in comp_cases:
    
    print('which projector=', comp)
    # errors:
    before_max = 0.
    before_L2  = 0.
    before_L1  = 0.

    before_max_fast = 0.
    before_L2_fast  = 0.
    before_L1_fast  = 0.
    
    for Nel_k in Nel_cases:
    
        # number of elements
        Nel = [3*Nel_k, 2*Nel_k, Nel_k]   
        print('Nel=', Nel)


        # element boundaries
        el_b = [np.linspace(0., L_i, Nel_i + 1) for L_i, Nel_i in zip(L, Nel)] 

        # knot sequences
        T = [bsp.make_knots(el_b_i, p_i, bc_i) for el_b_i, p_i, bc_i in zip(el_b, p, bc)] 
    
        # number of quadrature points per element
        Nq = [Nq_k, Nq_k, Nq_k]

        # create an instance of the projector class
        obj      = proj_glob.projectors_3d(T, p, bc, Nq)
        obj_fast = proj_glob_fast.projectors_3d(T, p, bc, Nq)
    

        # ---------------------------------------------
        # create LU decomposition of necessary matrices
        # ---------------------------------------------
        if comp==0:
            obj.NNN_LU()   # for PI_0  
        elif comp==11:
            obj.DNN_LU()  # for PI_11
        elif comp==12:
            obj.NDN_LU()  # for PI_12 
        elif comp==13:
            obj.NND_LU()  # for PI_13
        elif comp==21:
            obj.NDD_LU()  # for PI_21
        elif comp==22:
            obj.DND_LU()  # for PI_22
        elif comp==23:
            obj.DDN_LU()  # for PI_23
        elif comp==3:
            obj.DDD_LU()  # for PI_3

        # --------------------
        # compute coefficients
        # --------------------
        t0 = time.time()
        
        if comp==0:
            coeffs = obj.PI_0(fun)
        elif comp==11:
            coeffs = obj.PI_11(fun)
        elif comp==12:
            coeffs = obj.PI_12(fun)
        elif comp==13:
            coeffs = obj.PI_13(fun)
        elif comp==21:
            coeffs = obj.PI_21(fun)
        elif comp==22:
            coeffs = obj.PI_22(fun)
        elif comp==23:
            coeffs = obj.PI_23(fun)
        elif comp==3:
            coeffs = obj.PI_3(fun)
            
        t1 = time.time()
        print('time, original  =     %10.7e' % (t1-t0))
        t0 = time.time()
        
        # evaluate analytical function "fun_a" "fun_b" separately (as an example)
        #  and compute product at the points of projector
        mat_f_a = obj_fast.eval_for_PI(comp,fun_a)
        mat_f_b = obj_fast.eval_for_PI(comp,fun_b)
        mat_f = mat_f_a*mat_f_b        

        t1 = time.time()
        dt = t1-t0
        t0 = time.time()
        
        # get coefficients from 
        coeffs_fast = obj_fast.PI_mat(comp, mat_f  )

        t1 = time.time()
        print('time_fast, eval + integrate & project, %10.7e + %10.7e = %10.7e' % (dt,(t1-t0),dt+(t1-t0)))
        print('post processing...')

        # grid points for error computation
        pts_loc, wts_loc = np.polynomial.legendre.leggauss(5)   # quadrature points per element

        pts1, wts1 = bsp.quadrature_grid(el_b[0], pts_loc, wts_loc)   
        pts2, wts2 = bsp.quadrature_grid(el_b[1], pts_loc, wts_loc) 
        pts3, wts3 = bsp.quadrature_grid(el_b[2], pts_loc, wts_loc) 

        xgrid = [ pts1.flatten(), pts2.flatten(), pts3.flatten() ]   # error grid
        
        # projection evaluated at the grid points

        t0 = time.time()
        #evaluate matrix once
        f_basemat= eva.FEM_evalbase_3d(comp, xgrid, T, p, bc)
        t1 = time.time()
        print('time, eval FEM base     =     %10.7e' % (t1-t0))
        t0 = time.time()
        f_h      = kron_matvec_3d(f_basemat,coeffs)
        f_h_fast = kron_matvec_3d(f_basemat,coeffs_fast)
        t1 = time.time()
        print('time, eval 2xFEM fields =     %10.7e' % (t1-t0))
        
        t0 = time.time()
        # exact function evaluated at the grid points
        f = np.empty((xgrid[0].size, xgrid[1].size, xgrid[2].size))
        
        pts_1,pts_2,pts_3=np.meshgrid(xgrid[0],xgrid[1],xgrid[2],indexing='ij',sparse=True)
        f_exact = fun(pts_1,pts_2,pts_3)
        
        t1 = time.time()
        print('time, eval exact func  =     %10.7e' % (t1-t0))
        
        # max-norm
        err_max      = np.max( np.abs(f_exact - f_h) )
        err_max_fast = np.max( np.abs(f_exact - f_h_fast) )
        
        # L2-norm
        err_L2      = np.sqrt( np.sum( np.abs(f_exact - f_h)**2 ) / np.size(f) )
        err_L2_fast = np.sqrt( np.sum( np.abs(f_exact - f_h_fast)**2 ) / np.size(f) )

        # L1-norm
        err_L1      = np.sum( np.abs(f_exact - f_h) ) / np.size(f) 
        err_L1_fast = np.sum( np.abs(f_exact - f_h_fast) ) / np.size(f)

        # print errors
        print( 'orig,  Nq=', Nq,
               '{:10.7f}'.format(err_max), '{:10.7f}'.format(before_max/err_max), ' | ',
               '{:10.7f}'.format(err_L2),  '{:10.7f}'.format(before_L2/err_L2),   ' | ', 
               '{:10.7f}'.format(err_L1),  '{:10.7f}'.format(before_L1/err_L1),   ' | ')
        
        print( 'fast  ,Nq=', Nq,
               '{:10.7f}'.format(err_max_fast), '{:10.7f}'.format(before_max_fast/err_max_fast), ' | ',
               '{:10.7f}'.format(err_L2_fast),  '{:10.7f}'.format(before_L2_fast/err_L2_fast),   ' | ', 
               '{:10.7f}'.format(err_L1_fast),  '{:10.7f}'.format(before_L1_fast/err_L1_fast),   ' | ')

        before_max = err_max
        before_L2  = err_L2
        before_L1  = err_L1
        
        before_max_fast = err_max_fast
        before_L2_fast  = err_L2_fast
        before_L1_fast  = err_L1_fast        
        print('done.')
        
    print('')